In [1]:
import os 
import cv2
import numpy as np
from tqdm import tqdm
import glob
import natsort
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

D:\Anaconda3\envs\kimna\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
D:\Anaconda3\envs\kimna\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


### noisy image load

In [17]:
path1 = 'D:/Jupyter/kimna/yolo (2)/2. yolo_v5/data/x_ray/images/test_REALESRGAN/'
temp = natsort.natsorted(os.listdir(path1))

In [18]:
files = []
for i in range(50):
    files.append(path1 + temp[i])

In [19]:
images1 = []
for fl in tqdm(files):
    try:
        image = cv2.imread(fl, cv2.IMREAD_COLOR)
        images1.append(image)
    except:
        break
generate = np.array(images1)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 31.31it/s]


In [20]:
images1[1].shape

(1080, 1920, 3)

### clean image load

In [6]:
path2 = 'D:/Jupyter/kimna/yolo (2)/2. yolo_v5/data/x_ray/images/test_orginal_500/'
temp2 = natsort.natsorted(os.listdir(path2))

In [7]:
files_2 = []
for i in range(50):
    files_2.append(path2 + temp2[i])

In [8]:
images = []
for fl in tqdm(files_2):
    try:
        image = cv2.imread(fl, cv2.IMREAD_COLOR)
        images.append(image)
    except:
        break
clean = np.array(images)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:02<00:00, 20.82it/s]


In [9]:
images[1].shape

(1080, 1920, 3)

In [ ]:
for i in range(0,len(clean)):
    
    fig=plt.figure(figsize=(15, 5))
    
    ax = plt.subplot(131)
    plt.imshow(clean[i])
    
    ax = plt.subplot(132)
    plt.imshow(generate[i])
    plt.show()

In [10]:
y_true_list = []
for i in range(50):
    y_true = tf.image.convert_image_dtype(clean[i], tf.float32)
    y_true_list.append(y_true)

In [11]:
y_pred_list2 = []
for i in range(50):
    y_pred = tf.image.convert_image_dtype(generate[i], tf.float32)
    y_pred_list2.append(y_pred)

## PSNR

In [21]:
y_pred_list = []
for i in range(50):
    y_pred = tf.image.convert_image_dtype(generate[i], tf.float32)
    y_pred_list.append(y_pred)

In [22]:
#PSNR
psnr_list = []
for i in range(50):
    try:
        psnr_ouput = tf.image.psnr(y_true_list[i], y_pred_list[i], max_val=1.0)
        psnr_list.append(psnr_ouput)
    except:
        break

In [23]:
len(psnr_list)

50

In [24]:
psnr_list = np.array(psnr_list)

In [25]:
type(psnr_list)

numpy.ndarray

In [26]:
filePath = 'D:/Jupyter/kimna/5. project1/RESRGAN_PSNR.txt'

with open(filePath, 'w') as lf:
    for i in range(len(psnr_list)):
        lf.write(str(psnr_list[i]))
        lf.write('\n')

# SSIM

In [27]:
#SSIM
ssim_list = []
for i in range(len(y_true_list)):
    ssim_ouput = tf.image.ssim(y_true_list[i], y_pred_list[i], max_val=1.0)
    ssim_list.append(ssim_ouput)

In [28]:
ssim_list = np.array(ssim_list)

In [29]:
#SSIM
filePath = 'D:/Jupyter/kimna/5. project1/RESRGAN_SSIM.txt'

with open(filePath, 'w') as lf:
    for i in range(len(ssim_list)):
        lf.write(str(ssim_list[i]))
        lf.write('\n')

### 평가

In [ ]:
clean = clean*255
generate = generate*255

#ref_mat=patch
#res_mat=mask
#print(ref_mat.shape)
#print(res_mat.shape)

In [ ]:
def output_psnr_mse(img_orig, img_out):
    squared_error = np.square(img_orig - img_out)
    mse = np.mean(squared_error)
    psnr = 10 * np.log10(1.0 / mse)
    return psnr

In [ ]:
psnr_output = []
for i in range(len(clean)):
    output = output_psnr_mse(clean[i], generate[i])
    psnr_output.append(output)

In [ ]:
psnr_output

In [ ]:
ref_mat = ref_mat.astype('float') / 255.0
res_mat = res_mat.astype('float') / 255.0

def mean_psnr_srgb(ref_mat, res_mat):
    n_blk, h, w, c = ref_mat.shape
    mean_psnr = 0
    for b in range(n_blk):
        ref_block = ref_mat[b, :, :, :]
        res_block = res_mat[b, :, :, :]
        ref_block = np.reshape(ref_block, (h, w, c))
        res_block = np.reshape(res_block, (h, w, c))
        psnr = output_psnr_mse(ref_block, res_block)
        mean_psnr += psnr
    return mean_psnr / n_blk

#PSNR
mean_psnr = mean_psnr_srgb(ref_mat, res_mat)
print('mean_psnr:')
print(mean_psnr)

In [ ]:
from skimage.metrics import structural_similarity as ssim

def mean_ssim_srgb(ref_mat, res_mat):
    n_blk, h, w, c = ref_mat.shape
    mean_ssim = 0
    for b in range(n_blk):
        ref_block = ref_mat[b, :, :, :]
        res_block = res_mat[b, :, :, :]
        ref_block = np.reshape(ref_block, (h, w, c))
        res_block = np.reshape(res_block, (h, w, c))
        ssim1 = ssim(ref_block, res_block, gaussian_weights=True, use_sample_covariance=False,
                     multichannel=True)
        mean_ssim += ssim1
    return mean_ssim / n_blk

# SSIM
mean_ssim = mean_ssim_srgb(ref_mat, res_mat)
print('mean_ssim:')
print(mean_ssim)

In [ ]:
import shutil

### Haze

In [ ]:
HAZE_BEST_LIST = []

HAZE_BEST_LIST_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/HAZE_BEST_SSIM.txt'

f = open(HAZE_BEST_LIST_path, 'r')
for i in range(704):
    line = f.readline()
    HAZE_BEST_LIST.append(line.replace("\n",""))

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/14. inference_denoise_haze_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/21. best_haze_ssim/'

In [ ]:
for i in range(704):
    num = HAZE_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)

### Light

In [ ]:
LIGHT_BEST_LIST = []

LIGHT_BEST_PATH = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/LIGHT_BEST_SSIM.txt'

f = open(LIGHT_BEST_PATH, 'r')
for i in range(371):
    line = f.readline()
    LIGHT_BEST_LIST.append(line.replace("\n",""))

In [ ]:
LIGHT_BEST_LIST[0]

In [ ]:
len(LIGHT_BEST_LIST)

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/15. inference_denoise_light_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/22. best_light_ssim/'
    
for i in range(371):
    num = LIGHT_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)

### dark

In [ ]:
DARK_BEST_LIST = []

DARK_BEST_PATH = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/DARK_BEST_SSIM.txt'

f = open(DARK_BEST_PATH, 'r')
for i in range(114):
    line = f.readline()
    DARK_BEST_LIST.append(line.replace("\n",""))

In [ ]:
from_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/13. inference_denoise_dark_l/'
to_path = 'D:/Jupyter/Nahyoun/1. denoise/data/2. inference/3. inference_denosie/20. best_dark_ssim/'
    
for i in range(114):
    num = DARK_BEST_LIST[i]
    from_image_path = from_path+num+'.jpg'
    to_image_path = to_path+num+'.jpg'
    shutil.copyfile(from_image_path, to_image_path)

In [ ]:
import shutil

In [ ]:
import math
import numpy
import numpy.linalg
from scipy.special import gamma
from scipy.ndimage.filters import gaussian_filter
import scipy.misc
import scipy.io
import skimage.transform

"""
Generalized Gaussian distribution estimation.
Cite: 
Dominguez-Molina, J. Armando, et al. "A practical procedure to estimate the shape parameter in the generalized Gaussian distribution.", 
  available through http://www. cimat. mx/reportes/enlinea/I-01-18_eng. pdf 1 (2001).
"""

"""
Generalized Gaussian ratio function
Cite: Dominguez-Molina 2001, pg 7, eq (8)
"""
def generalized_gaussian_ratio(alpha):
    return (gamma(2.0/alpha)**2) / (gamma(1.0/alpha) * gamma(3.0/alpha))


"""
Generalized Gaussian ratio function inverse (numerical approximation)
Cite: Dominguez-Molina 2001, pg 13
"""
def generalized_gaussian_ratio_inverse(k):
    a1 = -0.535707356
    a2 = 1.168939911
    a3 = -0.1516189217
    b1 = 0.9694429
    b2 = 0.8727534
    b3 = 0.07350824
    c1 = 0.3655157
    c2 = 0.6723532
    c3 = 0.033834

    if k < 0.131246:
        return 2 * math.log(27.0/16.0) / math.log(3.0/(4*k**2))
    elif k < 0.448994:
        return (1/(2 * a1)) * (-a2 + math.sqrt(a2**2 - 4*a1*a3 + 4*a1*k))
    elif k < 0.671256:
        return (1/(2*b3*k)) * (b1 - b2*k - math.sqrt((b1 - b2*k)**2 - 4*b3*(k**2)))
    elif k < 0.75:
        #print "%f %f %f" % (k, ((3-4*k)/(4*c1)), c2**2 + 4*c3*log((3-4*k)/(4*c1)) )
        return (1/(2*c3)) * (c2 - math.sqrt(c2**2 + 4*c3*math.log((3-4*k)/(4*c1))))

"""
Estimate the parameters of an asymmetric generalized Gaussian distribution
"""
def estimate_aggd_params(x):
    x_left = x[x < 0]
    x_right = x[x >= 0]
    stddev_left = math.sqrt((1.0/(x_left.size - 1)) * numpy.sum(x_left ** 2))
    stddev_right = math.sqrt((1.0/(x_right.size - 1)) * numpy.sum(x_right ** 2))
    if stddev_right == 0:
        return 1, 0, 0 # TODO check this
    r_hat = numpy.mean(numpy.abs(x))**2 / numpy.mean(x**2)
    y_hat = stddev_left / stddev_right
    R_hat = r_hat * (y_hat**3 + 1) * (y_hat + 1) / ((y_hat**2 + 1) ** 2)
    alpha = generalized_gaussian_ratio_inverse(R_hat)
    beta_left = stddev_left * math.sqrt(gamma(3.0/alpha) / gamma(1.0/alpha))
    beta_right = stddev_right * math.sqrt(gamma(3.0/alpha) / gamma(1.0/alpha))
    return alpha, beta_left, beta_right

def compute_features(img_norm):
    features = []
    alpha, beta_left, beta_right = estimate_aggd_params(img_norm)

    features.extend([ alpha, (beta_left+beta_right)/2 ])

    for x_shift, y_shift in ((0,1), (1,0), (1,1), (1,-1)):
        img_pair_products  = img_norm * numpy.roll(numpy.roll(img_norm, y_shift, axis=0), x_shift, axis=1)
        alpha, beta_left, beta_right = estimate_aggd_params(img_pair_products)
        eta = (beta_right - beta_left) * (gamma(2.0/alpha) / gamma(1.0/alpha))
        features.extend([ alpha, eta, beta_left, beta_right ])

    return features

def normalize_image(img, sigma=7/6):
    mu  = gaussian_filter(img, sigma, mode='nearest')
    mu_sq = mu * mu
    sigma = numpy.sqrt(numpy.abs(gaussian_filter(img * img, sigma, mode='nearest') - mu_sq))
    img_norm = (img - mu) / (sigma + 1)
    return img_norm


def niqe(img):
    model_mat = scipy.io.loadmat('modelparameters.mat')
    model_mu = model_mat['mu_prisparam']
    model_cov = model_mat['cov_prisparam']

    features = None
    img_scaled = img
    for scale in [1,2]:

        if scale != 1:
            img_scaled = skimage.transform.rescale(img, 1/scale)
            #img_scaled = scipy.misc.imresize(img_norm, 0.5)

        # print img_scaled
        img_norm = normalize_image(img_scaled)

        scale_features = []
        block_size = 96//scale
        for block_col in range(img_norm.shape[0]//block_size):
            for block_row in range(img_norm.shape[1]//block_size):
                block_features = compute_features( img_norm[block_col*block_size:(block_col+1)*block_size, block_row*block_size:(block_row+1)*block_size] )
                scale_features.append(block_features)
        # print "len(scale_features)=%f" %(len(scale_features))
        if features == None:
            features = numpy.vstack(scale_features)
            # print features.shape
        else:
            features = numpy.hstack([features, numpy.vstack(scale_features)])
            # print features.shape
        
    features_mu = numpy.mean(features, axis=0)
    features_cov = numpy.cov(features.T)

    pseudoinv_of_avg_cov  = numpy.linalg.pinv((model_cov + features_cov)/2)
    niqe_quality = math.sqrt( (model_mu - features_mu).dot( pseudoinv_of_avg_cov.dot( (model_mu - features_mu).T ) ) )

    return niqe_quality

In [ ]:
run -i niqe.py

In [ ]:
niqe(y_pred_list[1])